In [1]:
#This version of the evolutionary algorithm seeks to implement
#step 8, i.e. the possiblity for converting factories
#also look at different objective functions

#Interestingly enough, adding production efficiency results in the most
#efficient strategy for 'rushing' being to produce only military factories

In [2]:
#Production efficiency techs take 150 days base, construction and industry take 200 base

#Meta strat:
#21 march 1936: Improved worker conditions
#2 september 1936: War eco
#27 november 1936: Popular figurehead
#30 march 1937: Captain of industry, state 12


#Normal strat:
#17 july 1936: Spanish civil war, war eco


#Research:
#23 may 1936: Basic machine tools, state 4
#8 july 1936: Construction 1, state 5
#24 november 1936: Concentrated 1, state 9 
#28 feburari 1937: Construction 2, state 12
#11 april 1937: Improved machine tools, state 13
#25 may 1937: Concentrated 2, state 14

#ASSUMPTION FOR NOW: All further tech is unlocked 3 months after they become current TEST IN GAME
#Concentrated 3, Machine tools 3, Construction 3, state 33
#Concentrated 4, Machine tools 4, Construction 4, state 54

In [3]:
#STABILITY EVENTS:
#State 2: +5%
#State 4: +5%
#State 8: +10%

In [4]:
import numpy as np

In [5]:
def selection(pop, scores, k=3):
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        if scores[ix] > scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if np.random.rand() < r_cross:
        pt = np.random.randint(1, len(p1)-1)
        #print(pt)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
        #visualise what solutions have been tried
    return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if np.random.rand() < r_mut:
            bitstring[i] = 1 - bitstring[i]
    return bitstring

In [6]:
def update_consumergoods(state, toaster_percent):
    match state:
        case 5:
            toaster_percent -= 0.15 #WAR ECO        
    return round(toaster_percent, 2)

def update_constructionspeed(state, civ_output_modifier):
    match state:
        case 0:
            civ_output_modifier += 0.15
        case 5:
            civ_output_modifier += 0.1
        case 12:
            civ_output_modifier += 0.2 #tech + captain of industry
        case 33:
            civ_output_modifier += 0.1
        case 54:
            civ_output_modifier += 0.1
    return round(civ_output_modifier, 2)

def update_miloutput(state, mil_output_modifier):
    match state:
        case 9:
            mil_output_modifier += 0.15
        case 14:
            mil_output_modifier += 0.15
        case 33:
            mil_output_modifier += 0.15
        case 54:
            mil_output_modifier += 0.15
    return round(mil_output_modifier, 2)

def update_prodefficiency(state, production_efficiency_cap):
    match state:
        case 4:
            production_efficiency_cap += 0.1
        case 13:
            production_efficiency_cap += 0.1
        case 33:
            production_efficiency_cap += 0.1
        case 54:
            production_efficiency_cap += 0.1
    return round(production_efficiency_cap, 2)

def update_civcost(state, civ_construction_modifier):
    match state:
        case 5:
            civ_construction_modifier += 0.3
    return round(civ_construction_modifier, 2)

def update_milcost(state, mil_construction_modifier):
    match state:
        case 5:
            mil_construction_modifier += 0.5
    return round(mil_construction_modifier, 2)

In [7]:
#Two max_targets: 'rush', 'scale', 'scale_total' and 'scale_civs'

def production(actions, max_target):
    mils = 32
    mils_0 = 32 #keep track of the number of starting factories
    civs = 45
    civs_0 = 45 #idem
    total = mils + civs
    total_0 = 32 + 45 #idem
    consumergoods_percentage = 0.40
    consumergoods = np.floor(total * consumergoods_percentage)
    available_civs = np.maximum(int(civs - consumergoods), 0)
    
    #Civ output modifiers are additive (-30% + 15% = -15%), multiplied by state infrastructure
    civ_output = 5 
    civ_output_modifier = 1 #Construction speed
    civ_construction_modifier = 0.7
    mil_construction_modifier = 0.7
    mil_output = 4.5
    mil_output_modifier = 1
    prod_eff = 0.5
    production_efficiency_cap = 0.55
    
    #Investigate how new factories add to production efficiency
    #production_efficiency works as on the wiki, except for the starting factories
    #with different rules
    #The number of factories because of trade is quite significant, but ultimately unpredictable
    
    state = 0
    state_size_days = 35 #how many days per state
    ic_produced = 0
    ic_total = 0    
    civ_purchase_pool = 0 
    mil_purchase_pool = 0
    civ_to_mil_pool = 0
    mil_to_civ_pool = 0
    infrastructure_modifier = 1.6
    
    for i in range(len(actions)):
        consumergoods_percentage = (update_consumergoods(i, consumergoods_percentage))
        civ_output_modifier = (update_constructionspeed(i, civ_output_modifier))
        mil_output_modifier = (update_miloutput(i, mil_output_modifier))
        civ_construction_modifier = (update_civcost(i, civ_construction_modifier))
        mil_construction_modifier = (update_milcost(i, mil_construction_modifier))
        production_efficiency_cap = (update_prodefficiency(i, production_efficiency_cap))
        
        ic_produced = mil_output * mils * state_size_days * mil_output_modifier * prod_eff
        
        if actions[i] == 0: #build mils     
            mil_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_construction_modifier)
            
            existing_mils = mils #Used exclusively for production efficiency calculations
            mils += np.floor_divide(mil_purchase_pool, 7200) 
            total += np.floor_divide(mil_purchase_pool, 7200) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            mil_purchase_pool = np.mod(mil_purchase_pool, 7200)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            prod_eff = (existing_mils * prod_eff + (mils - existing_mils) * 0.1) / mils
            growth = 0.001 * (production_efficiency_cap **2 / prod_eff)
            prod_eff += (growth * state_size_days) / 2
            prod_eff = min(1, prod_eff)
            
            state += 1
            ic_total += ic_produced
            
        if actions[i] == 1: #build civs         
            civ_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_construction_modifier)
            civs += np.floor_divide(civ_purchase_pool, 10800) 
            total += np.floor_divide(civ_purchase_pool, 10800) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            civ_purchase_pool = np.mod(civ_purchase_pool, 10800)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            growth = 0.001 * (production_efficiency_cap **2 / prod_eff)
            prod_eff += (growth * state_size_days) / 2
            prod_eff = min(1, prod_eff)
            
            state += 1
            ic_total += ic_produced

        if actions[i] == 2: #Convert civs to mils
            civ_to_mil_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_construction_modifier)

            civs -= np.floor_divide(civ_to_mil_pool, 4000)
            mils += np.floor_divide(civ_to_mil_pool, 4000)
            civ_to_mil_pool = np.mod(civ_to_mil_pool, 4000)
            consumergoods = np.ceil(total * consumergoods_percentage)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
        
        if actions[i] == 3: #Convert mils to civs
            mil_to_civ_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_construction_modifier)
            if mils > 0:
                civs += np.floor_divide(mil_to_civ_pool, 9000)
                mils -= np.floor_divide(mil_to_civ_pool, 9000)
                
                if mils < 0:
                    too_many_removed = abs(mils)
                    civs -= too_many_removed
                    mils = 0
                    mil_to_civ_pool = 0
                
                else:
                    mil_to_civ_pool = np.mod(mil_to_civ_pool, 9000)
                consumergoods = np.ceil(total * consumergoods_percentage)
                available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
            
    match max_target:
        case 'rush':
            return (ic_total, mils, civs)
        case 'scale':
            scaled_score = ic_total * np.log2((mils/mils_0))
            return (scaled_score, mils, civs, ic_total)
        case 'scale_total':
            scaled_score = ic_total * np.log2((mils/mils_0)) * (total/total_0)
            if scaled_score == None:
                scaled_score = 0
            return (scaled_score, mils, civs, ic_total)
        case 'scale_civs':
            scaled_score = ic_total * np.log2((mils/mils_0)) * np.log2((civs/civs_0))
            if scaled_score == None:
                scaled_score = 0
            return (scaled_score, mils, civs, ic_total)

In [8]:
#production([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
production([3] * 20, 'scale')

(-77607.76170446104, 9.0, 68.0, 42406.875)

In [9]:
test = [0] * 57
test2 = [1] * 16 + [0] * 41
test3 = [1] * 57

production(test, 'rush'), production(test2, 'rush'), production(test3, 'rush')

#production(test, 'scale'), production(test2, 'scale'), production(test3, 'scale')

#production(test, 'scale_total'), production(test2, 'scale_total'), production(test3, 'scale_total')

#production(test, 'scale_civs'), production(test2, 'scale_civs'), production(test3, 'scale_civs')

((530856.7617686796, 93.0, 45),
 (490066.75858166564, 103.0, 57.0),
 (316115.34711203014, 32, 161.0))

In [10]:
def genetic_algorithm(n_bits, n_pop, n_iter, r_cross, max_target):
    pop = [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
    best, best_eval = 0, production(pop[0], max_target)
    
    for gen in range(n_iter):
        scores = [production(c, max_target) for c in pop]
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                print("Generation " + str(gen) + ": new best solution found!")
                print(best, best_eval)
                
        #ADD ELITISM PLUS ADDITIONAL COMPLETELY RANDOM SAMPLES HERE
        
        selected = [selection(pop, scores) for _ in range(n_pop)]
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            c = crossover(p1, p2, r_cross)
            for child in c:
                mutated = mutation(child, 1 / n_bits)
                children.append(mutated)
        #print(np.mean(sum([production(c) for c in pop])))
        #print(np.mean(sum([production(c) for c in children])))
        #print(sum([sum(strat) for strat in pop]))
        pop = children
        
    #SMOOTHING TEST
    best_smoothed = [best[0]]
    for i in range(1, len(best)-1):
        #print((best[i-1],best[i],best[i+1]))
        if best[i-1] == best[i+1]:
            best_smoothed.append(best[i-1])
        else:
            best_smoothed.append(best[i])
    best_smoothed.append(best[-1])
    best_smoothed_eval = production(best_smoothed, max_target)
    if best_smoothed_eval > best_eval:
        print("Smoothing : new best solution found!")
        print(best_smoothed, best_smoothed_eval)
        return [best_smoothed, best_smoothed_eval]
    
    return [best, best_eval]

In [11]:
genetic_algorithm(10,10, 5, 0.9, 'scale') #There should be 66 states of 30 days, ends up on June 5th '41
#Note: states could also be 35 days to reflect the length of the focus tree
#This would be 57 states of 35 days, ending onJune 19th '41 (no leap years)

Generation 0: new best solution found!
[0, 0, 0, 1, 1, 1, 0, 1, 1, 0] (4842.574370278196, 36.0, 47.0, 28498.304129320226)
Generation 0: new best solution found!
[0, 0, 1, 0, 0, 0, 1, 0, 0, 0] (7093.253650511348, 38.0, 46.0, 28610.19155151321)
Generation 2: new best solution found!
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0] (9234.48797454326, 40.0, 45.0, 28684.939653292033)
Generation 3: new best solution found!
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (9273.55921344215, 40.0, 45, 28806.306006584557)


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 (9273.55921344215, 40.0, 45, 28806.306006584557)]

In [12]:
#smoothing does not bring better results anymore, at least for 'scale' max_target
#the results for 'scale' seem very similar to those of 'rush'
#It produces 96% of the IC, has 6% more mils and 13% more civs
genetic_algorithm(57,100, 30, 0.9, 'scale')

Generation 0: new best solution found!
[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1] (683055.6424962741, 90.0, 73.0, 457857.1738576398)
Generation 0: new best solution found!
[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0] (684970.7035087948, 86.0, 68.0, 480254.9465312026)
Generation 0: new best solution found!
[1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0] (719178.5553620599, 92.0, 67.0, 472037.6171792174)
Generation 0: new best solution found!
[0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1] (72

[[1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 (828872.9238929104, 100.0, 53.0, 504224.7181041426)]

In [13]:
#The best strategy for 'scale_total' produces about 81% percent of the IC that the rush strategy does
#It ends up with 18% extra mils and 66% extra civs though
genetic_algorithm(57,100, 30, 0.9, 'scale_total')

Generation 0: new best solution found!
[0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0] (1388023.3760707928, 87.0, 75.0, 457217.8382647879)
Generation 0: new best solution found!
[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1] (1423384.6152088328, 89.0, 78.0, 444721.9912722919)
Generation 0: new best solution found!
[1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0] (1446128.1107843039, 90.0, 74.0, 455121.74448834383)
Generation 0: new best solution found!
[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1]

[[1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 (1829389.5272074256, 109.0, 71.0, 442585.1850810074)]

In [14]:
#'scale_civs' produces 67% of 'rush' IC, but has 4% more mils and 142% more civs
genetic_algorithm(57,100, 30, 0.9, 'scale_civs')

Generation 0: new best solution found!
[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1] (576186.8134270243, 89.0, 84.0, 433599.6139805276)
Generation 1: new best solution found!
[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0] (603171.2081249338, 81.0, 96.0, 411832.66520011437)
Generation 3: new best solution found!
[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1] (610812.0396912828, 84.0, 94.0, 412804.1347160748)
Generation 3: new best solution found!
[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0] (6

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 (735432.6466780206, 98.0, 109.0, 356849.43740796106)]

In [15]:
genetic_algorithm(57,100, 30, 0.9, 'rush')

Generation 0: new best solution found!
[0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1] (490060.4999437283, 88.0, 62.0)
Generation 0: new best solution found!
[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1] (491267.09102463693, 77.0, 68.0)
Generation 0: new best solution found!
[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1] (494996.35418076155, 88.0, 60.0)
Generation 1: new best solution found!
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1] (507005.7556808925, 86.0, 56.0)
Generation 4: new best sol

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1],
 (530822.8564156725, 91.0, 45.0)]

In [16]:
#Building civs for the first 16 states (roughly 1.5 years) seems to be it's optimal solution
#Building civs for the first 12 states (roughly 1.25 years) seems to also be an optimal solution
#Interestingly, building civs for 13, 14 or 15 performs worse than either, ergo the solution space has local maxima
#How do you account for the fact that the second solution has better long-term potential?
#How can I scale the factory scores as an added weight?
#Check in game if the difference of four states really equals 7 civilian factories

max_12 = production([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'scale')
max_16 = production([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'scale')
max_12, max_16

((824976.9339618754, 99.0, 52.0, 506320.669026752),
 (826497.8467029501, 103.0, 57.0, 490066.75858166564))

In [17]:
production([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1])

TypeError: production() missing 1 required positional argument: 'max_target'

In [ ]:
base_efficiency = 0.5
efficiency_cap = 0.6
new_efficiency = 0.1
mils = 10
new_mils = 1
total_mils = mils + new_mils
efficiency = round((base_efficiency * mils + new_mils * new_efficiency) / total_mils, 4)
efficiency_start = efficiency

for i in range(35):
    growth = 0.001 * (efficiency_cap **2 / efficiency)
    efficiency += growth
    print(round(efficiency, 4), round(growth, 7))
    
print(round(efficiency - efficiency_start, 4))

In [ ]:
growth = 0.001 * (efficiency_cap **2 / efficiency)
efficiency += (growth * 35) / 2
    
print(round(efficiency - efficiency_start, 4))